In [ ]:
##
# This is the comment format of the section that may need to be modified
##

Up load the data file to google colab

In [ ]:
#https://colab.research.google.com/notebooks/io.ipynb
# Upload file to google colab
# use to upload test .scv file
from google.colab import files
uploaded = files.upload()

Saving mydata.csv to mydata.csv


Load csv file and store into an array with pandas

In [ ]:
#https://pandas.pydata.org/docs/reference/general_functions.html
import pandas as pd

#Load the data and store the data into a array.

##
# the name of the .csv file which contains the data of features.
##
data_file = 'mydata.csv'

#load the csv file as array
features = pd.read_csv(data_file)

# check the information of the array
# .shape    https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html
# .describe https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
# .head     https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

#features.shape
#features.describe()
#features.head(3)

Transforming data into a more program-readable format

In [ ]:
# Convert categorical variable into dummy/indicator variables.
#.get_dummies https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
##
# Actual data may require a different approach to processing
##
features = pd.get_dummies(features)

#features.shape
#features.describe()
features.head(3)

,year,month,day,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,43,50,44,29,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,41,50,44,61,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,43,46,47,56,0,0,0,1,0,0,0


In [ ]:
# Use numpy to convert to arrays
# numpy has more built-in mathematical formulas than pandas, not necessary
# https://numpy.org/install/
import numpy as np

# now the data = features_1 + features_2 + ...+ features_n + result
# we spride it into two parts  "features_1 + features_2 + ...+ features_n" => featues and "result" => result

# Cut the results from the current data and store into the array called lables
labels = np.array(features['actual'])

# Remove the labels from the old array
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
# axis is 1 refers to the columns
features= features.drop('actual', axis = 1)
feature_list = list(features.columns)

# Convert the panda array to numpy array
features = np.array(features)

In [ ]:
# https://scikit-learn.org/stable/ 
# Using Skicit-learn to segment data
# One part is the training data
# Another part is the test data
from sklearn.model_selection import train_test_split

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
##
# split_size represents the ratio of training data to test data
# 0.25 means 75% is training data, 25% is test data
# can be replace by function RepeatedStratifiedKFold (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html)
##
split_size = 0.25

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = split_size, random_state = 42)

In [ ]:
#print('Training Features Shape:', train_features.shape)
#print('Training Labels Shape:', train_labels.shape)
#print('Testing Features Shape:', test_features.shape)
#print('Testing Labels Shape:', test_labels.shape)

In [ ]:
#print('Training Features :', train_features)
#print('Training Labels :', train_labels)
#print('Testing Features :', test_features)
#print('Testing Labels :', test_labels)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# Create RandomForest model by using the function from sklearn
import sklearn
#print(sklearn.__version__)
from sklearn.ensemble import RandomForestRegressor

# n_estimatorsint, default=100
# random_stateint, RandomState instance or None, default=None
random_froest_model = RandomForestRegressor(n_estimators = 100, random_state = None)

# Train the model with training data
random_froest_model.fit( train_features , train_labels );

# https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html
# Use the model we trained to test the test_freautures
# and compare the result with the real test_labels
mode_result = random_froest_model.predict( test_features )

# The error is the abosult difference between mode_result and test_labels
errors = abs( mode_result - test_labels )

# calculate the accuracy of the model
accuracy = 100 - np.mean( 100 * ( errors / test_labels ) )
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 93.96 %.


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html
# This function generates a GraphViz representation of the decision tree, 
#  which is then written into out_file. Once exported, graphical renderings can be generated using, for example:
from sklearn.tree import export_graphviz
# https://tmilan0604.medium.com/visualisation-with-pydot-for-beginners-ca99c9dc530b
import pydot

# Pull out one tree from the forest
tree = random_froest_model.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'randomtree.dot', feature_names = feature_list)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('randomtree.dot')
# save the graph as png
graph.write_png('randomtree.png')